In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import collections

<h2> Recognizing Bipartite-Class Graphs </h2>

In [3]:
#Assume input of the form V, E where V = {v1,v2,v3,...} or the set of all vertices, and E = {{vi,vj},...} as the set 
#of all edges in the graph
V = {1,2,3,4,5,6,7}
E = {(1,2),(2,3),(3,4),(4,1),(5,6),(6,7)}

In [4]:
def single_parser(NB, A, B):
    rem = dict()
    for key in NB.keys():
        if A[key]==True:
            for neigh in NB[key]:
                B[neigh] = True
                if (A[neigh]==True):
                    return rem, False
        elif B[key]==True:
            for neigh in NB[key]:
                A[neigh] = True
                if (B[neigh]==True):
                    return rem, False
        else:
            rem[key] = NB[key]
    return rem, True
def connected_parser(NB):
    A = dict(zip(list(V), np.repeat(False, len(V))))
    B = A.copy()
    #Set starting point
    A[list(NB.keys())[0]] = True
    rem, state = single_parser(NB, A, B)
    if not state:
        return rem, state
    if len(rem)>0:
        rem, state = single_parser(rem, A, B)
    return rem, state

In [5]:
def bipartite_recog(V, E):
    #Re-shape to a dictionary of neighbourhoods:
    NB = dict()
    for edge in E:
        for key, value in [(0,1),(1,0)]:
            if not NB.get(edge[key], None):
                NB[edge[key]] = collections.deque([edge[value]])
            else:
                NB[edge[key]].append(edge[value])
    state = True
    while (len(NB)>0):
        NB, valid = connected_parser(NB)
        state = state and valid
        if not state:
            return state
    return state

In [6]:
bipartite_recog(V, E)

True

<h2> Hamilton Cycle Recognition </h2>

In [7]:
def hamilton_help(start, end, seen, V, NB):
    if (end in NB[start]) and (len(seen)>=2) and (set(seen).union(set([start]))==V):
        seen.append(start)
        seen.append(end)
        return seen, True
    else:
        seen.append(start)
        for adj in set(NB[start])-set(seen):
            seq, bl = hamilton_help(adj, end, seen.copy(), V, NB)
            if bl:
                return seq, bl
        return None, False

In [12]:
def hamilton_recog(V,E):
    #Re-shape to a dictionary of neighbourhoods:
    NB = dict()
    for edge in E:
        for key, value in [(0,1),(1,0)]:
            if not NB.get(edge[key], None):
                NB[edge[key]] = collections.deque([edge[value]])
            else:
                NB[edge[key]].append(edge[value])
    return hamilton_help(list(V)[0], list(V)[0], collections.deque(), V, NB)

In [13]:
V = {1,2,3,4,5,6,7}

In [14]:
E = {(1,2),(1,3),(2,3),(3,4),(4,5),(5,1),(5,6),(6,7),(7,1)}

In [15]:
hamilton_recog(V,E)

(deque([1, 2, 3, 4, 5, 6, 7, 1]), True)